# Spark and Elasticsearch Integration

<b>Este bloco de notas demonstra como você pode usar o Spark para ler dados de streaming, realizar uma transformação nos dados e gravar os novos dados no Elasticsearch.

Também demonstra como você pode usar o Spark para ler um índice do Elasticsearch.
</b>

# Instalando o ES-Hadoop

<b>Primeiro, precisamos instalar o conector Elastic-hadoop. 
Isso deve ser instalado no mesmo caminho em todo o cluster.</b>

In [42]:
import os
import urllib.request
import zipfile

In [2]:
print(os.listdir('../../file/'))


['.bash_history', '.cache', '.ipynb_checkpoints', '.local', 'elasticsearch-hadoop-7.5.2', 'Untitled.ipynb']


# Obtendo a configuração do Elasticsearch

In [4]:
os.chdir('../../file/')

#!pip install elasticsearch


In [5]:
from elasticsearch import Elasticsearch

<b> Primeiro configure a conexão do Elasticsearch por padrão, nos conectamos ao elasticsearch: 9200, como estamos executando este notebook no Spark-Node, precisamos usar 'elasticsearch' em vez de 'localhost', pois esse é o nome do docker ontainer executando o Elasticsearch. Se o índice de teste de fluxo existir, limpe-o e crie um novo. </b>

In [6]:
es = Elasticsearch('elasticsearch:9200')

if es.indices.exists('stream-test'):
    es.indices.delete('stream-test')
    es.indices.create('stream-test')

# Spark Streaming para Elasticsearch

<b> Precisamos garantir que o conector ES-Hadoop esteja no caminho de classe do driver. </b>

In [9]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import json
import time
from datetime import datetime

print(os.getcwd())

import os  
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars elasticsearch-hadoop-7.5.2/dist/elasticsearch-spark-20_2.11-7.5.2.jar pyspark-shell'

/file


<b> Agora podemos criar nosso Contexto Spark. </b>

In [10]:
sc = SparkContext(appName="PythonSparkStreaming")  
sc.setLogLevel("WARN")

<b>...e nosso contexto de streaming</b>

In [11]:
ssc = StreamingContext(sc, 3)

<b> Em seguida, vamos gerar um fluxo de arquivos. Nesse caso, transmitiremos todos os arquivos gravados no diretório de amostra. Isso está sendo bombeado com dados aleatórios.</b>

In [12]:
stream = ssc.textFileStream('sample/')

<b>Vamos querer realizar algumas transformações leves nesses dados. Primeiramente, queremos analisar o horário da época em uma sequência de datas que o Elasticsearch possa entender.</b>

In [13]:
def format_sample(x):
    data = json.loads(x)
    data['timestamp'] = datetime.fromtimestamp(data['timestamp']).strftime('%Y/%m/%d %H:%M:%S')
    data['doc_id'] = data.pop('count')
    return (data['doc_id'], json.dumps(data))

<b>Mapearemos a função que acabamos de criar para o fluxo, para que cada registro, em cada lote, seja analisado com a mesma função.</b>

In [14]:
parsed = stream.map(lambda x: format_sample(x))

<b> Em seguida, vamos definir uma função que grave o RDD gerado por cada operação em lote de streaming no Elasticsearch. </b>

In [15]:
def handler(rdd):
        es_write_conf = {
        # especifique o nó para o qual estamos enviando dados (esse deve ser o mestre)
        "es.nodes" : 'elasticsearch',
            
        # especifique a porta caso ela não seja a porta padrão
        "es.port" : '9200',
            
        # especifique um recurso no formato 'index/doc-type'
        "es.resource" : 'stream-test/sample',

        # e a entrada JSON?
        "es.input.json" : "yes",
            
        # existe um campo no mapeamento que deve ser usado para especificar o ID do documento ES
        "es.mapping.id": "doc_id",
        }

        rdd.saveAsNewAPIHadoopFile(
                path='-',
                outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",
                keyClass="org.apache.hadoop.io.NullWritable",
                valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",

                # criticamente, devemos especificar nosso 'es_write_conf' 
                conf=es_write_conf)

<b>Agora podemos aplicar nosso manipulador a cada registro transmitido pelo sistema.</b>

In [16]:
parsed.foreachRDD(lambda rdd: handler(rdd))

<b>Podemos solicitar que também seja impresso em 'stdout'.</b>

In [17]:
parsed.pprint()

<b>Finalmente, podemos começar o contexto da centelha.</b>

In [18]:
ssc.start()

-------------------------------------------
Time: 2020-01-30 18:30:39
-------------------------------------------

-------------------------------------------
Time: 2020-01-30 18:30:42
-------------------------------------------

-------------------------------------------
Time: 2020-01-30 18:30:45
-------------------------------------------

-------------------------------------------
Time: 2020-01-30 18:30:48
-------------------------------------------



<b>Se queremos parar o contexto, precisamos chamar o método 'stop'</b>

In [19]:
ssc.stop()

# Processamento em massa ES com Spark (Bulk)

In [20]:
sc = SparkContext(appName="PythonSparkReading")  
sc.setLogLevel("WARN")

In [33]:
es_read_conf = { 
    # especifique o nó para o qual estamos enviando dados (esse deve ser o mestre)    
    "es.nodes" : "elasticsearch",
    
    # especifique o recurso de leitura no formato 'index / doc-type'
    "es.resource" : "stream-test"
    }



In [34]:
es_rdd = sc.newAPIHadoopRDD(
    inputFormatClass="org.elasticsearch.hadoop.mr.EsInputFormat",
    keyClass="org.apache.hadoop.io.NullWritable", 
    valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable", 
    conf=es_read_conf)
    

<b>Vamos pegar uma amostra dos dados que acabamos de extrair do Elasticsearch. Observe que cada objeto é uma tupla, onde o primeiro item é o ID do documento.</b>

In [37]:

es_rdd.take(1)

[]

<b>Vamos converter essas tuplas em JSON puro</b>

In [45]:
es_rdd = es_rdd.map(lambda x: x[1])

In [46]:
es_rdd.take(1)

[]

<b>Agora, vamos converter o RDD em um Dataframe Spark SQL para que possamos tratá-lo mais como um objeto Pandas</b>

In [47]:
from pyspark.sql import SparkSession, SQLContext, Row

# Executa um 'monkey patch' no contexto do Spark, estendendo-o com recursos do Spark SQL
spark = SparkSession \
    .builder \
    .appName("Spark SQL") \
    .getOrCreate()

In [48]:
df = es_rdd.map(lambda l: Row(**dict(l))).toDF()

ValueError: RDD is empty

In [ ]:
df.take(1)

<b> Podemos executar um 'groupby' nos dados. Nesse caso, agruparemos por nome </b>

In [ ]:
df \
    .groupby('name') \
    .count() \
    .collect()

<b>Também podemos filtrar dados como no 'Pandas'.</b>

In [ ]:
df \
    .filter(df.name == 'Samwise')\
    .take(1)

<b>Novamente, precisamos parar o contexto, se quisermos voltar para um Contexto de Streaming</b>

In [ ]:
sc.stop()